In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import re
import requests
import math
from bs4 import BeautifulSoup
import matplotlib as mlp
import matplotlib.pyplot as plt
import matplotlib.backends.backend_agg
import matplotlib.figure
import seaborn as sb
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.metrics import roc_curve, roc_auc_score

In [2]:
'''listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=2').content, 'lxml')
listings_soup.find_all('h4', {'class': 'heading-3 margin-y-1 font-weight-bold'})'''
page1_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/').content, 'lxml')
page1_prices_soup = page1_listings_soup.find_all('h4', {'data-test': 'vehicleCardPricingBlockPrice'})
page1_prices_soup

[<h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$12,651</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$36,995</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$38,995</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$7,995</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$26,500</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$26,419</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$7,995</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$30,475</h4>,
 <h4 class

In [3]:
pricespage1 = re.findall('[0-9]+,[0-9]+', str(page1_prices_soup))
pricespage1

['12,651',
 '36,995',
 '38,995',
 '7,995',
 '26,500',
 '26,419',
 '7,995',
 '30,475',
 '28,300',
 '7,991',
 '9,991',
 '9,888',
 '8,799',
 '12,924',
 '32,977',
 '31,945',
 '30,998',
 '9,999',
 '10,099',
 '30,680',
 '21,997',
 '26,600',
 '11,622',
 '32,011',
 '24,995',
 '8,000',
 '29,999',
 '29,950',
 '9,000',
 '27,550',
 '26,988',
 '27,888',
 '13,000']

In [4]:
page1_prices = []
for price in pricespage1:   
    page1_prices.append(str(price).replace(',', '.'))
page1_prices

['12.651',
 '36.995',
 '38.995',
 '7.995',
 '26.500',
 '26.419',
 '7.995',
 '30.475',
 '28.300',
 '7.991',
 '9.991',
 '9.888',
 '8.799',
 '12.924',
 '32.977',
 '31.945',
 '30.998',
 '9.999',
 '10.099',
 '30.680',
 '21.997',
 '26.600',
 '11.622',
 '32.011',
 '24.995',
 '8.000',
 '29.999',
 '29.950',
 '9.000',
 '27.550',
 '26.988',
 '27.888',
 '13.000']

In [5]:
#Scraping rest of pages listings prices

other_pages_prices1 = []
for number in range(2, 190):
    page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(number)).content, 'lxml')
    page_prices_soup = page_listings_soup.find_all('h4', {'data-test': 'vehicleCardPricingBlockPrice'})
    page_prices = list(re.findall('[0-9]+,[0-9]+', str(page_prices_soup)))
    other_pages_prices1 += page_prices
other_pages_prices1

['27,950',
 '9,900',
 '4,999',
 '9,991',
 '8,000',
 '10,500',
 '25,488',
 '18,499',
 '14,985',
 '14,933',
 '7,995',
 '10,000',
 '6,290',
 '12,995',
 '14,648',
 '9,785',
 '8,495',
 '5,995',
 '7,990',
 '6,752',
 '31,674',
 '19,000',
 '33,207',
 '6,991',
 '14,970',
 '12,888',
 '16,791',
 '24,895',
 '3,999',
 '14,419',
 '8,999',
 '23,590',
 '11,000',
 '5,495',
 '30,990',
 '13,999',
 '11,998',
 '11,995',
 '10,999',
 '25,235',
 '11,491',
 '34,980',
 '10,995',
 '4,297',
 '5,999',
 '26,995',
 '11,994',
 '22,995',
 '5,499',
 '8,995',
 '22,577',
 '22,950',
 '7,490',
 '10,889',
 '8,379',
 '4,288',
 '11,000',
 '9,250',
 '8,980',
 '18,995',
 '25,995',
 '28,583',
 '1,500',
 '8,990',
 '9,995',
 '5,300',
 '23,613',
 '4,595',
 '26,795',
 '8,995',
 '6,995',
 '13,500',
 '28,500',
 '16,000',
 '2,499',
 '11,990',
 '14,495',
 '28,750',
 '12,900',
 '14,990',
 '5,987',
 '10,991',
 '10,387',
 '6,991',
 '4,500',
 '2,699',
 '2,850',
 '28,799',
 '13,000',
 '8,618',
 '11,979',
 '11,779',
 '4,382',
 '8,989',
 '10,4

In [6]:
rest_pages_prices1 = []
for price in other_pages_prices1:   
    rest_pages_prices1.append(price.replace(',', '.'))
rest_pages_prices1

['27.950',
 '9.900',
 '4.999',
 '9.991',
 '8.000',
 '10.500',
 '25.488',
 '18.499',
 '14.985',
 '14.933',
 '7.995',
 '10.000',
 '6.290',
 '12.995',
 '14.648',
 '9.785',
 '8.495',
 '5.995',
 '7.990',
 '6.752',
 '31.674',
 '19.000',
 '33.207',
 '6.991',
 '14.970',
 '12.888',
 '16.791',
 '24.895',
 '3.999',
 '14.419',
 '8.999',
 '23.590',
 '11.000',
 '5.495',
 '30.990',
 '13.999',
 '11.998',
 '11.995',
 '10.999',
 '25.235',
 '11.491',
 '34.980',
 '10.995',
 '4.297',
 '5.999',
 '26.995',
 '11.994',
 '22.995',
 '5.499',
 '8.995',
 '22.577',
 '22.950',
 '7.490',
 '10.889',
 '8.379',
 '4.288',
 '11.000',
 '9.250',
 '8.980',
 '18.995',
 '25.995',
 '28.583',
 '1.500',
 '8.990',
 '9.995',
 '5.300',
 '23.613',
 '4.595',
 '26.795',
 '8.995',
 '6.995',
 '13.500',
 '28.500',
 '16.000',
 '2.499',
 '11.990',
 '14.495',
 '28.750',
 '12.900',
 '14.990',
 '5.987',
 '10.991',
 '10.387',
 '6.991',
 '4.500',
 '2.699',
 '2.850',
 '28.799',
 '13.000',
 '8.618',
 '11.979',
 '11.779',
 '4.382',
 '8.989',
 '10.4

In [7]:
other_pages_prices2 = []

for number in range(190, 305):
    page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(number)).content, 'lxml')
    page_prices_soup = page_listings_soup.find_all('h4', {'data-test': 'vehicleCardPricingBlockPrice'})
    page_prices = list(re.findall('[0-9]+,[0-9]+', str(page_prices_soup)))
    other_pages_prices2 += page_prices
    
rest_pages_prices2 = []

for price in other_pages_prices2:   
    rest_pages_prices2.append(price.replace(',', '.'))

In [8]:
prices = page1_prices + rest_pages_prices1 + rest_pages_prices2
len(prices)

9093

In [ ]:
#Scraping rest of pages listings prices

other_pages_prices1 = []
for number in range(2, 190):
    page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(number)).content, 'lxml')
    page_prices_soup = page_listings_soup.find_all('h4', {'data-test': 'vehicleCardPricingBlockPrice'})
    page_prices = list(re.findall('[0-9]+,[0-9]+', str(page_prices_soup)))
    other_pages_prices1 += page_prices
other_pages_prices1

In [ ]:
rest_pages_prices1 = []
for price in other_pages_prices1:   
    rest_pages_prices1.append(price.replace(',', '.'))
rest_pages_prices1

In [28]:
def pageyears(pages):
    if pages == 1:
        fst_page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/').content, 'lxml')
        fst_page_listings = fst_page_listings_soup.find_all('span', {'class': 'vehicle-card-year'})
        fst_page_years = list(re.findall('[12][0-9]{3}', str(fst_page_listings)))
        return fst_page_years
    else:
        nth_page_listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(pages)).content, 'lxml')
        nth_page_listings = nth_page_listings_soup.find_all('span', {'class': 'vehicle-card-year'})
        nth_page_years = list(re.findall('[12][0-9]{3}', str(nth_page_listings)))
        return nth_page_years

In [32]:
page_years1 = []
for number_of_page in range(1, 250):
    page_years1 += pageyears(number_of_page)
print(len(page_years))
page_years1

9093


['2017',
 '2014',
 '2017',
 '2016',
 '2018',
 '2016',
 '2017',
 '2018',
 '2018',
 '2019',
 '2018',
 '2018',
 '2017',
 '2017',
 '2017',
 '2018',
 '2018',
 '2016',
 '2017',
 '2016',
 '2016',
 '2016',
 '2019',
 '2017',
 '2017',
 '2016',
 '2017',
 '2018',
 '2017',
 '2018',
 '2017',
 '2016',
 '2017',
 '2017',
 '2018',
 '2017',
 '2012',
 '2018',
 '2017',
 '2018',
 '2019',
 '2017',
 '2017',
 '2017',
 '2014',
 '2017',
 '2015',
 '2017',
 '2017',
 '2014',
 '2016',
 '2017',
 '2015',
 '2019',
 '2017',
 '2018',
 '2019',
 '2017',
 '2016',
 '2017',
 '2012',
 '2017',
 '2017',
 '2017',
 '2017',
 '2016',
 '2018',
 '2017',
 '2017',
 '2015',
 '2016',
 '2019',
 '2017',
 '2012',
 '2016',
 '2018',
 '2017',
 '2016',
 '2017',
 '2015',
 '2017',
 '2017',
 '2018',
 '2013',
 '2017',
 '2014',
 '2005',
 '2017',
 '2018',
 '2018',
 '2017',
 '2014',
 '2017',
 '2018',
 '2017',
 '2017',
 '2015',
 '2016',
 '2015',
 '2017',
 '2017',
 '2016',
 '2016',
 '2017',
 '2002',
 '2017',
 '2017',
 '2017',
 '1999',
 '2017',
 '2014',
 

In [ ]:
page_years2 = []
for number_of_page in range(250, 500):
    page_years2 += pageyears(number_of_page)
print(len(page_years2))
page_years2

In [ ]:
page_years = list(map(pageyears, list(range(1, 100))))
page_years